<div class="alert alert-block alert-info"> <b>NOTE</b> Please select the kernel <code>Python [conda env: deepcell]</code> for this notebook. </div>

# 1. Inference

In [2]:
import os
import sys
import timeit

import numpy as np
import tensorflow as tf

from deepcell.applications import NuclearSegmentation
from deepcell_tracking.trk_io import load_trks, save_trk
from deepcell_tracking.tracking import CellTracker as _CellTracker

sys.path.append('/publication-tracking/benchmarking')
import utils

In [ ]:
# Add timers to CellTracker
class CellTracker(_CellTracker):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
        # Add timers
        self.timer_ne = 0
        self.timer_inf = 0
        self.timer_lap = 0

    def _get_neighborhood_embeddings(self, **kwargs):
        tic = timeit.default_timer()
        embeddings = super()._get_neighborhood_embeddings(**kwargs)
        toc = timeit.default_timer()

        self.timer_ne += toc - tic

        return embeddings

    def _track_frame(self, frame):
        tic = timeit.default_timer()
        cost_matrix, predictions = self._get_cost_matrix(frame)
        toc = timeit.default_timer()

        self.timer_inf += toc - tic

        tic = timeit.default_timer()
        row_ind, col_ind = linear_sum_assignment(cost_matrix)
        assignments = np.stack([row_ind, col_ind], axis=1)
        self._update_tracks(assignments, frame, predictions)
        toc = timeit.default_timer()

        self.timer_lap += toc - tic


In [5]:
source_data = '/publication-tracking/data/test.trks'

data_dir = '/publication-tracking/benchmarking/DeepCell/data'
gt_seg_dir = os.path.join(data_dir, 'SEG_GT')
pred_seg_dir = os.path.join(data_dir, 'SEG_PRED')

for d in [data_dir, gt_seg_dir, pred_seg_dir]:
    if not os.path.exists(d):
        os.makedirs(d)
        
model_urls = {
    'NuclearSegmentation': 'https://deepcell-data.s3-us-west-1.amazonaws.com/saved-models/NuclearSegmentation-75.tar.gz',
    'NuclearTrackingNE': 'https://deepcell-data.s3-us-west-1.amazonaws.com/saved-models/NuclearTrackingNE-75.tar.gz',
    'NuclearTrackingInf': 'https://deepcell-data.s3-us-west-1.amazonaws.com/saved-models/NuclearTrackingInf-75.tar.gz'
}

In [6]:
# Load test data
data = load_trks(source_data)

In [7]:
# Download and load each model
models = {}
for m, url in model_urls.items():
    archive_path = tf.keras.utils.get_file(f'{m}.tgz', url, extract=True, cache_subdir='models')
    model_path = os.path.splitext(archive_path)[0]
    model = tf.keras.models.load_model(model_path)
    models[m] = model

450641920/450633493 [==============================] - 37s 0us/step


2022-11-28 23:20:21.717644: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-28 23:20:23.396179: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10396 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:05:00.0, compute capability: 6.1


548864/538726 [==============================] - 0s 0us/step


In [8]:
# Load segmentation and tracking applications
app_seg = NuclearSegmentation(models['NuclearSegmentation'])

In [ ]:
timers = []

In [9]:
for batch_no in range(len(data['lineages'])):
    print(batch_no)
    tm = {}
    
    # Pull out relevant data for this batch
    X = data['X'][batch_no]
    y = data['y'][batch_no]
    lineage = data['lineages'][batch_no]
    
    # Correct discontiguous tracks, which are not allowed by CTC
    y, lineage = utils.convert_to_contiguous(y, lineage)

    # Determine position of zero padding for removal
    slc = utils.find_zero_padding(X)
    X = X[slc]
    y = y[slc]

    # Determine which frames are zero padding
    frames = np.sum(y, axis=(1,2)) # True if image not blank
    good_frames = np.where(frames)[0]
    X = X[:len(good_frames)]
    y = y[:len(good_frames)]
    
    # Generate tracks on predicted segmentations
    tic = timeit.default_timer()
    y_pred = app_seg.predict(X)
    tm['seg'] = timeit.default_timer() - tic

    tracker = CellTracking(
        X,
        y_pred,
        models['NuclearTrackingInf'],
        neighborhood_encoder=models['NuclearTrackingNE'],
        track_length=8
    )
    tracker.track_cells()
    tm['ne'] = tracker.timer_ne
    tm['inf'] = tracker.timer_inf
    tm['lap'] = tracker.timer_lap

    tm['nframes'] = X.shape[0]
    tm['nobjects'] = ...

    timers.append(tm)

0


2022-11-28 23:21:48.627508: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100
/usr/local/lib/python3.8/dist-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0. Please use `footprint` instead.
  markers = h_maxima(image=maxima,


1


/usr/local/lib/python3.8/dist-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0. Please use `footprint` instead.
  markers = h_maxima(image=maxima,


2


...

/usr/local/lib/python3.8/dist-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0. Please use `footprint` instead.
  markers = h_maxima(image=maxima,


10


/usr/local/lib/python3.8/dist-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0. Please use `footprint` instead.
  markers = h_maxima(image=maxima,


11


/usr/local/lib/python3.8/dist-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0. Please use `footprint` instead.
  markers = h_maxima(image=maxima,


# 2. Evaluation

In [10]:
import glob
import os
import re
import subprocess

import numpy as np
import pandas as pd

from deepcell_tracking.metrics import TrackingMetrics

In [11]:
data_dir = '/publication-tracking/benchmarking/DeepCell/data'
gt_seg_dir = os.path.join(data_dir, 'SEG_GT')
pred_seg_dir = os.path.join(data_dir, 'SEG_PRED')

pattern = re.compile('\d{3}_GT')
data_ids = [f.split('_')[0] for f in os.listdir(gt_seg_dir) if pattern.fullmatch(f)]

node_match_threshold = 0.6

ctc_software = '/publication-tracking/benchmarking/CTC_Evaluation_Software'
operating_system = 'Linux' # or 'Mac' or 'Win'
num_digits = '3'

In [12]:
benchmarks = []

for results_dir, s in zip([gt_seg_dir, pred_seg_dir], ['GT', 'Deepcell']):
    for data_id in data_ids:
        results = {
            'model': f'Deepcell - {s}',
            'data_id': os.path.splitext(data_id)[0]
        }
        gt_dir = os.path.join(results_dir, f'{data_id}_GT/TRA')
        res_dir = os.path.join(results_dir, f'{data_id}_RES')
        
        # Deepcell benchmarking
        m = TrackingMetrics.from_isbi_dirs(gt_dir, res_dir, threshold=node_match_threshold)
        results.update(m.stats)
        
        # CTC metrics
        for metric, path in [('DET', 'DETMeasure'), ('SEG', 'SEGMeasure'), ('TRA', 'TRAMeasure')]:
            p = subprocess.run([os.path.join(ctc_software, operating_system, path), results_dir, data_id, num_digits],
                               stdout=subprocess.PIPE)
            outstring = p.stdout
            
            try:
                val = float(outstring.decode('utf-8').split()[-1])
                results[metric] = val
            except:
                print('Benchmarking failure', path, results_dir, data_id)
                print(outstring.decode('utf-8'))
        
        benchmarks.append(results)

df = pd.DataFrame(benchmarks)
df.to_csv('benchmarks.csv')

missed node 1_29 division completely
missed node 29_29 division completely
missed node 57_10 division completely
missed node 60_0 division completely
18_16 out degree = 2, daughters mismatch, gt and res degree equal.
missed node 5_6 division completely
missed node 121_42 division completely
missed node 144_28 division completely
missed node 26_25 division completely
missed node 1_29 division completely
missed node 3_23 division completely
29_29 out degree = 2, daughters mismatch, gt and res degree equal.
corrected division 3_23 as a frameshift division not an error
missed node 10_19 division completely
15_17 out degree = 1, daughters mismatch.
missed node 19_20 division completely
29_17 out degree = 1, daughters mismatch.
missed node 60_34 division completely
missed node 76_7 division completely
corrected division 10_19 as a frameshift division not an error
corrected division 60_34 as a frameshift division not an error
corrected division 76_7 as a frameshift division not an error
misse

In [ ]:
df